## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-27-14-33-58 +0000,nypost,Pennsylvania principal axed after ranting abou...,https://nypost.com/2025/12/27/us-news/pennsylv...
1,2025-12-27-14-33-00 +0000,nypost,"Hakeem Jeffries’ aides, NY lawmakers went on S...",https://nypost.com/2025/12/27/us-news/private-...
2,2025-12-27-14-29-52 +0000,wapo,Russian attack pummels Kyiv as Zelensky prepar...,https://www.washingtonpost.com/world/2025/12/2...
3,2025-12-27-14-18-07 +0000,nypost,Ilhan Omar’s hubby’s $30M firm quietly scrubs ...,https://nypost.com/2025/12/27/us-news/ilhan-om...
4,2025-12-27-14-17-41 +0000,nyt,Saudi-led Group in Yemen Tells Separatists to ...,https://www.nytimes.com/2025/12/27/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,25
115,year,12
191,new,10
27,meet,9
303,one,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...,75
135,2025-12-26-21-22-50 +0000,nypost,Trump tells Post there’s a ‘good shot’ at reac...,https://nypost.com/2025/12/26/us-news/trump-te...,72
74,2025-12-27-07-34-22 +0000,nyt,Zelensky Will Meet With Trump Over the Weekend...,https://www.nytimes.com/2025/12/26/world/europ...,67
67,2025-12-27-08-44-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,66
36,2025-12-27-11-15-43 +0000,cbc,Zelenskyy to meet Carney in Halifax today befo...,https://www.cbc.ca/news/carney-zelenskyy-meeti...,61


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,75,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...
45,51,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
135,47,2025-12-26-21-22-50 +0000,nypost,Trump tells Post there’s a ‘good shot’ at reac...,https://nypost.com/2025/12/26/us-news/trump-te...
90,34,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...
82,30,2025-12-27-05-01-06 +0000,nyt,"Before This Physicist Studied the Stars, He Wa...",https://www.nytimes.com/2025/12/27/world/europ...
111,30,2025-12-26-23-49-29 +0000,bbc,Two killed in suspected Palestinian ramming an...,https://www.bbc.com/news/articles/c9d9znp22pyo...
3,28,2025-12-27-14-18-07 +0000,nypost,Ilhan Omar’s hubby’s $30M firm quietly scrubs ...,https://nypost.com/2025/12/27/us-news/ilhan-om...
103,28,2025-12-27-01-27-11 +0000,nypost,Americans in line for ‘gigantic’ tax refund ea...,https://nypost.com/2025/12/26/us-news/scott-be...
34,27,2025-12-27-11-29-00 +0000,startribune,This all-seasons biker tracks Minneapolis Chai...,https://www.startribune.com/minneapolis-chain-...
18,23,2025-12-27-12-58-48 +0000,nypost,Jeffrey Epstein’s shocking Amazon purchase his...,https://nypost.com/2025/12/27/us-news/jeffrey-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
